# Advent of Code 2018, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202018%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [5]:
⍝ Helper functions and common settings
⎕IO←0
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
lines←{⊃⎕NGET ⍵ 1}
ilines←{⍎¨lines ⍵}
line←{⊃lines ⍵}
sorted←{⍵[⍋⍵]}
sortbycol←⊢⌷⍨∘⊂∘⍒⌷⍤1
pairs←{↓(2÷⍨≢⍵) 2⍴⍵}
]box on -style=max -trains=tree -fns=on
]rows on

┌→─────────────────────────────────────┐
│Was ON -style=min -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→──────┐
│Was OFF│
└───────┘

### Day 1: Chronal Calibration
https://adventofcode.com/2018/day/1

In [39]:
DAY1←ilines'data/2018/01.txt'

In [87]:
⊢Part1←+/DAY1
assert 477=Part1

477

For part 2 we're looking for the first frequency to recur. If we get to the end, take the last value and use that as the start value for the next iteration. We look for the first duplicate using APL's [unique mask](https://aplwiki.com/wiki/Nub_Sieve) function.     

In [32]:
Day1p2←{t←⍬⋄0{t,←1↓+\⍺,⍵⋄1=≢e←t[⍸<\~≠t]:⊃e⋄⍵∇⍨⊃¯1↑t}⍵}

In [86]:
⊢Part2←Day1p2 DAY1
assert 390=Part2

390

### Day 2: Inventory Management System
https://adventofcode.com/2018/day/2

In [85]:
⊢Part1←×/+/⍉↑2 3∘∊¨(≢⊢)⌸¨DAY2←lines'data/2018/02.txt'
assert 8118=Part1

8118

In [84]:
⊢Part2←(=⌿l)/0⌷l←↑DAY2[⍸{1∊⍵}¨↓DAY2∘.{+/≠/⍺,⍪⍵}DAY2]
assert 'jbbenqtlaxhivmwyscjukztdp'≡Part2

┌→────────────────────────┐
│jbbenqtlaxhivmwyscjukztdp│
└─────────────────────────┘

### Day 3: No Matter How You Slice It
https://adventofcode.com/2018/day/3

In [128]:
DAY3←⍎¨¨'\d+'⎕s'&'¨lines'data/2018/03.txt'

In [152]:
R←1000 1000⍴0⋄_←{R[⍵]+←1}¨CL←{,((2⊃⍵)+⍳4⊃⍵)∘.,((1⊃⍵)+⍳3⌷⍵)}¨DAY3 ⍝ Convert origin + dx dy to coord pairs

In [153]:
⊢Part1←+/2≤∊R
assert 104126=Part1

104126

In [154]:
⊢Part2←1+⊃⍸{∧/1=R[⍵]}¨CL ⍝ Claim numbers are 1-indexed, so need to add 1
assert 695=Part2          

695

### Day 4: Repose Record
https://adventofcode.com/2018/day/4

Basically a series of set operations -- sort based on date, then split and merge into chunks based on each guard's log lines. Pair up into sleep-wake pairs and expand into the corresponding minute range. We can then use key `⌸` to find the most frequent minute for the guard with the most sleep minutes.

The regex

   '(?|.*(#\d+).*|.*:(\d\d)).*'
   
is a _branch reset_ `(?| ...|... )`, which means that the result of whichever of the two branches that matches can be referred to as `\1` in the replacement.

In [393]:
'iotag'⎕CY'dfns'
DAY4←sorted lines'data/2018/04.txt'

In [394]:
parsed←'(?|.*(#\d+).*|.*:(\d\d)).*'⎕r'\1'⊢DAY4 ⍝ Pick out guard number, and minutes for each sleep/wake item

In [395]:
guards←{0⊃0⊃⍵}¨merged←(≠{⍎1↓0⊃⍵}¨grouped)⊂grouped←sorted parsed⊂⍨'#'∘(1∊⍷)¨parsed

In [396]:
sleepiest←⊃⍒≢¨sleep←{∊{(⍎⍺) iotag ¯1+⍎⍵}/↑⍵}¨pairs¨{⊃,/{1↓⍵}¨⍵}¨merged

In [397]:
⊢Part1←(⍎1↓sleepiest⊃guards)×⊃((⊣/⊢⍤/⍨∘(⌈/=⊢)⊢/){⍺(≢⍵)}⌸)sleepiest⊃sleep
assert 138280=Part1

138280

For part 2, we seek the guard which has the highest minute sleep frequency. We have most of the parts already.

In [412]:
f←⊃⍒{⌈/1⌷⍉⍵}¨freq←({⍺,(≢⍵)}⌸)¨sleep

In [417]:
⊢Part2←(⍎1↓f⊃guards)×⊃(⊣/⊢⍤/⍨∘(⌈/=⊢)⊢/)f⊃freq
assert 89347=Part2

89347

### Day 5: Alchemical Reduction
https://adventofcode.com/2018/day/5

In [ ]:
DAY5←line'data/2018/05.txt'

In [418]:
r←∊'|'(1↓∘,,⍤0)p,⌽¨p←(⎕C⎕A),¨⎕A ⍝ Make the regex aA|Aa|bB|Bb ... etc

In [433]:
⊢Part1←≢{r⎕r''⊢⍵}⍣≡⊢DAY5        ⍝ Apply regex replace until no change
assert 9172=Part1

9172

In [435]:
⊢Part2←⌊/{≢{r⎕r''⊢⍵}⍣≡⊢(⍵⎕r''⍠'IC'1)DAY5}¨⎕A ⍝ This several minutes to run
assert 6550=Part2

6550

### Day 6: Chronal Coordinates
https://adventofcode.com/2018/day/6

This problem is very suited to an array language like APL. We create a 3D matrix where each major cell is a 2D matrix holding the manhattan distances from each point to the input coordinate. We can then process this along the z-axis to pick the layer index of the smallest distance. That gives us a "Nearest" map. What remains is to remove any "infinites", defined as the numbers which occur along any edge of the "Nearest" map, and finally find the highest frequency. 

The Python [solution](https://github.com/xpqz/aoc-18/blob/master/day6.py) is considerably longer and slower.

In [128]:
⎕io←0
DAY6←⍎¨lines'data/2018/06.txt'

In [129]:
(XMAX YMAX)←⊃↓⍉↑⌈/DAY6

For each of the 50 points, make an array where each value is the MHD from the point.

In [130]:
MHD←{+/|⍵-⍺}
Dist←{YMAX XMAX⍴⍵∘MHD¨,⍳YMAX XMAX}

Create the nearest distance map. A small hoop to jump through: points that are equidistant to several coordinates should be removed. We set those to ¯1.

In [131]:
Nearest←{v←⍋⍵⋄⍵[v[0]]=⍵[v[1]]:¯1⋄⊃v}¨↓[0]↑Dist¨⌽¨DAY6 ⍝ Flip each input pair so we have (y x)

Remove any items touching the boundaries of the array, as they're 'infinite'.

In [132]:
⊢Part1←1⊃0⌷1 sortbycol{⍺(≢⍵)}⌸(∊Nearest)~∪Nearest[⍸(⌽∨⊖)0∊¨⍳⍴Nearest]
assert 3989=Part1

3989

For part 2, we instead seek the set of points that each have a combined MHD to all coords < 10,000.

In [133]:
⊢Part2←+/{10000>+/⍵∘MHD¨DAY6}¨,⍳YMAX XMAX
assert 49715=Part2

49715

### Day 7: The Sum of Its Parts
https://adventofcode.com/2018/day/7

We have a graph which we need to traverse in a specific child-order - sorting the queue each iteration is a somewhat inefficient approach, but as the number of nodes is tiny it doesn't really matter.

In [105]:
DAY7←'^.+?([A-Z]).*([A-Z]).*$'⎕r'\1\2'⊢lines'data/2018/07.txt'

In [106]:
(KEY PRE)←↓⍉↑(↓⊃¨,∘⊂⌸⊢/¨)↓⌽↑DAY7 ⍝ Make the graph - splitting out nodes and their ascendants

In [107]:
Day7p1←{0=≢⍵:⍺⋄(⍺,p)∇(⍺,p)~⍨sorted ⍵∪{KEY/⍨⍵∘{∧/⍵∊⍺}¨PRE} ⍺,p←⊃⍵}

In [108]:
⊢Part1←⍬ Day7p1 sorted KEY~⍨∪∊DAY7
assert 'BHMOTUFLCPQKWINZVRXAJDSYEG'≡Part1

┌→─────────────────────────┐
│BHMOTUFLCPQKWINZVRXAJDSYEG│
└──────────────────────────┘

Part 2 -- process items concurrently in a worker pool of 5. Each job now takes 60 + a number corresponding to the item's position in the alphabet (A=60+1, B=60+2 etc). Replaces 200+ lines of my Python [solution](https://github.com/xpqz/aoc-18/blob/master/day7.py).

In [110]:
Workers←{0=slots←5-≢⍺:⍺⋄⍺,{⍵,⍨61+⎕A⍳⍵}¨(slots⌊≢⍵)↑⍵}

In [111]:
]dinput
Day7p2←{
    wrk←⍬ ⋄ sec←0
    ⍬ {
        (0=≢⍵)∧0=≢wrk:sec
        (remains jobs)←↓⍉↑wrk⊢←¯1∘+@0¨wrk Workers ⍵ ⍝ Fill any available slots and 'tick'
        wrk⊢←(0≠remains)/wrk                        ⍝ Reap any dead worker processes
        sec+←1
        (⍺,sorted ready)∇(⍺,jobs)~⍨⍵∪{KEY/⍨⍵∘{∧/⍵∊⍺}¨PRE}⍺,ready←(0=remains)/jobs
    } ⍵
}

In [112]:
⊢Part2←Day7p2 sorted KEY~⍨∪∊DAY7
assert 877=Part2

877

### Day 8: Memory Maneuver
https://adventofcode.com/2018/day/8

Release the recursions!

In [99]:
DAY8←⍎line'data/2018/08.txt'

In [100]:
get←{v←DAY8[CUR+⍳⍵]⋄CUR+←⍵⋄v}

In [101]:
Day8p1←{(chc mtc)←get 2⋄ch←⍬ {⍵=0:⍺⋄(⍺,Day8p1⍬)∇⍵-1} chc⋄⊂ch (get mtc)}

In [102]:
CUR←0
⊢Part1←+/∊TREE←Day8p1⍬
assert 44838=Part1

44838

In [103]:
Day8p2←{(ch mt)←⍵⋄0=≢ch:+/mt⋄ind←¯1+mt~0⋄+/∇¨ch[(ind<≢ch)/ind]}

In [104]:
⊢Part2←Day8p2 ⊃TREE
assert 22198=Part2

22198